In [1]:
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt 

pd.set_option('display.max_columns', None)  
from thesis_lib.data_processing import *
from thesis_lib.visual import *
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta

In [2]:
db = load_parquet('parquet_data')

Loading dataset:  laboratory
	 3086085
Loading dataset:  images
	 350403
Loading dataset:  hospitalizations
	 88256
Loading dataset:  surgeries
	 66750
Loading dataset:  sectors_admissions
	 154481
Loading dataset:  hospital_sectors
	 36
Formating integer columns
Formating date columns
Formating time columns


,operating_room,surgery_date,surgery_weekday,surgery_year_month,surgery_scheduled_time,admission_id,surgery_type,G.A.P.,surgery_id,origin,patient_id,gender,age,discharge_type,entity_description,diagnosis,scheduled_surgery,actual_surgery,surgery_physician,dependency,anesthesia_type,ASA,antisepsia,prophylactic_ATB,dosis_mg,bed_request,hemotherapy,hemo_ok,x_ray,cardiologist,supplies,supplies_ok,protocol_no,service_description,surgery_code,sector_bed,specialization,estimated_duration,surgery_startime,surgery_endtime,surgery_delay,hospitalization_date,specialization_code,entry_time,exit_time,anesthesia_startime,anesthesia_endtime,post_surgery_condition,admission_date,admission_time,discharge_date,discharge_time,re_surgery,no_of_surgeries,hips_surgery,injury_condition,pre_surgery_duration,surgery_duration,post_surgery_duration,surgery_prep_duration,re_admission_id,antibiotic,seeding,no._of_assistans,anesthetist_id,no._of_pregnancies,no._births,no._of_cesarean,nulliparous,new_born_id_rel,new_born_admission_id,new_born_weight,new_born_gestation_age,new_born_alive,bact_positive
0,7,2017-01-01,Dom,201701,21:00:00,466496-1,No Programada,False,137489,URG,654803-6,F,43,Alta Medica,OSPIT - O.S. PERS. IND. TEXTIL,APENDICITIS AGUDA,"APENDICECTOMIA NOCTURNA, LAPAROSCOPICA","APENDICECTOMIA NOCTURNA, LAPAROSCOPICA",SOTELO JUAN MANUEL,Interno,General,1.0,IODOPOVIDONA,AMPICILINA-SULBACTAM,1500.0,False,False,False,False,False,False,False,283049,CIRUGIA GENERAL,58052403,0,CIRUGIA GENERAL ADULTOS,00:34:00,21:41:00,22:15:00,30,None,PROC,21:09:00,22:34:00,21:35:00,22:23:00,INT,2017-01-01,14:51:00,2017-01-02,19:08:00,0.0,1,False,Limpia,00:06:00,00:34:00,00:19:00,00:25:00,None,-1.0,-1.0,1,56356,0,0,0,None,NaN,None,NaN,0,None,NaN


In [380]:
db['surgeries'][db['surgeries'].patient_id == '3526141-4']

,operating_room,surgery_date,surgery_weekday,surgery_year_month,surgery_scheduled_time,admission_id,surgery_type,G.A.P.,surgery_id,origin,patient_id,gender,age,discharge_type,entity_description,diagnosis,scheduled_surgery,actual_surgery,surgery_physician,dependency,anesthesia_type,ASA,antisepsia,prophylactic_ATB,dosis_mg,bed_request,hemotherapy,hemo_ok,x_ray,cardiologist,supplies,supplies_ok,protocol_no,service_description,surgery_code,sector_bed,specialization,estimated_duration,surgery_startime,surgery_endtime,surgery_delay,hospitalization_date,specialization_code,entry_time,exit_time,anesthesia_startime,anesthesia_endtime,post_surgery_condition,admission_date,admission_time,discharge_date,discharge_time,re_surgery,no_of_surgeries,hips_surgery,injury_condition,pre_surgery_duration,surgery_duration,post_surgery_duration,surgery_prep_duration,re_admission_id,antibiotic,seeding,no._of_assistans,anesthetist_id,no._of_pregnancies,no._births,no._of_cesarean,nulliparous,new_born_id_rel,new_born_admission_id,new_born_weight,new_born_gestation_age,new_born_alive,bact_positive


In [4]:
hospitalizations = db['hospitalizations']
hospitalizations.head()

,admission_id,patient_id,age,gender,birth_date,insurance_entity,entity_group,admission_date,admission_time,admission_year,admission_month,origin,admission_physician,admission_sector,last_sector,last_category,isolation,last_room,last_bed,discharge_date,discharge_time,discharge_year,discharge_month,discharge_reason,pre_discharge_date,first_sector,administrative_diagnosis,diagnosis_code,presumptive_dianogsis,discharge_diagnosis_code,discharge_diagnosis,date_registered_discharge,time_registered_discharge,discharge_physician,discharge_summary,discharge_summary_physician,surgery,express_hip_surgery,admission_lenght_days,responsible_sector,second_responsible_sector,emergency_admission_datetime,emergency_service,has_previous_admission,previous_admission_id,previous_admission_date,previous_discharge_date,previous_sector,previous_discharge_dianosis,discharge_ambulance,new_born_weight,new_born_gestation_age,PIM2TEP,high_risk_TEP,low_risk_TEP,ARM_TEP,CEC_TEP,request_number,request_origin,request,request_diagnosis,request_sector,notified,request_user
0,466470-1,3526141-4,79,F,1937-11-15,PAMI UGL VI,PAMI,2017-01-01,02:42:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,T16,C,None,1634,2,2017-01-04,18:17:00,2017,Ene,Alta Medica,NaT,T16,DESHIDRATACION,XX2,No Especifica Codigo,276.51,Deshidratacion,2017-01-04,16:47:41,SIMONE MILENA,True,SIMONE MILENA,False,False,3,CLINICA MEDICA,None,NaT,NaN,False,None,NaT,NaT,None,None,True,NaN,0,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,None
1,466471-4,3714597-2,74,F,1942-09-13,PAMI UGL VI,PAMI,2017-01-01,02:48:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,UC6,A,*,702,1,2017-01-07,14:03:00,2017,Ene,Alta Medica,NaT,UCO,DIF RESPIRATORIA,XX2,No Especifica Codigo,401.9,Hipertension Arterial No Especificada,2017-01-07,13:42:01,GIARROCCO AGUSTINA,True,GIARROCCO AGUSTINA,False,False,6,CARDIOLOGIA ADULTOS,None,NaT,NaN,False,None,NaT,NaT,None,None,False,NaN,0,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,None
2,466472-7,3714595-2,27,F,1989-11-08,MEDICUS,PREPAGA,2017-01-01,02:46:00,2017,Ene,Guardia,QUIROZ CARINA MAGDALENA,T10,T10,C,None,1020,1,2017-01-01,19:54:00,2017,Ene,Alta Medica,NaT,T10,ABORTO ESPONTANEO,XX2,No Especifica Codigo,634,Aborto Espontaneo,2017-01-01,19:30:56,ABADIA MICAELA,True,ABADIA MICAELA,True,False,1,OBSTETRICIA,None,NaT,NaN,False,None,NaT,NaT,None,None,False,NaN,0,NaN,NaN,NaN,None,None,NaN,None,None,None,None,None,None
3,466473-0,3672378-8,85,F,1931-08-09,OSUTHGRA O.S PERSONAL GASTRONOMICO,BRISTOL,2017-01-01,03:55:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,T12,C,None,1236,1,2017-01-04,14:58:00,2017,Ene,Alta Medica,NaT,T12,OCLUSION INTESTINAL,789.00,Dolor Abdominal Sitio Inespecificado,620.2,Otros Quistes Ovaricos Y Quister Ovaricos Neom,2017-01-04,14:25:44,CHAVEZ MORA JORGE MARIO,True,CHAVEZ MORA JORGE MARIO,False,False,3,CLINICA MEDICA,None,NaT,NaN,False,None,NaT,NaT,None,None,False,NaN,0,NaN,NaN,NaN,None,None,42736.059525,MESA OPERATIVA,DOM,OCLUSION INTESTINAL,INTERNACION GENERAL,SUP,ABLANCO
4,466474-3,3714598-2,34,F,1982-04-06,OSDE,PREPAGA,2017-01-01,05:07:00,2017,Ene,Emergencias,CANIS DIEGO DAMIAN,EME,EME,I,None,6,1,2017-01-01,09:01:00,2017,Ene,Alta Medica,NaT,EME,INTOX. ALCOHOLICA,XX2,No Especifica Codigo,94.68,Desintoxicacion Combinada De Alcohol Y Drogas,2017-01-01,09:01:03,PUENTE MOSQUERA KAROLA,True,PUENTE MOSQUERA KAROLA,False,False,1,None,None,NaT,NaN,False,None,NaT,NaT,None,None,False,NaN,0,NaN,NaN,NaN,None,None,42736.195347,ANDREA,DOMICILIO,INTOX. ALCOHOLICA,INTERNACION GENERAL,None,ABLANCO


In [6]:
for i in hospitalizations[hospitalizations['patient_id'] == '3526141-4'].index:
    print(i)

0
33624


In [233]:
from datetime import datetime
class Patient:
    def __init__(self, patient_id):
        self.patient_id = patient_id

    def load_patient_data(self,db):
        df = db['hospitalizations']
        self.hospitalizations_data = df[df['patient_id'] == self.patient_id].sort_values(by='admission_date')
        
        df = db['laboratory']
        self.laboratory_data = df[df['patient_id'] == self.patient_id].sort_values(by='admission_date')
       
        df = db['images']
        self.images_data = df[df['patient_id'] == self.patient_id].sort_values(by='admission_date')
    
    @property
    def admission_history(self): 
        
        admission_key = namedtuple('admission_key', ['admission_number','admission_id'])
        
        if hasattr(self, 'hospitalizations_data'):
            return {admission_key(admission_number,admission_id) : Admission(admission_id, self) 
                   for admission_number, admission_id in enumerate(self.hospitalizations_data.admission_id)}
        
        else:
            print('Load patient data before')
    
    def get_historic_records(self):

        records_history = pd.DataFrame()
        
        for admission in self.admission_history:
            admission_records = self.admission_history[admission].get_admission_records()
            if records_history.empty:
                records_history = admission_records
            else:
                records_history  = records_history.append(admission_records, ignore_index=True)
        
        return records_history
            

In [373]:
from collections import namedtuple
class Admission():
    def __init__(self, admission_id, patient):
        self.admission_id = admission_id
        self.admission_data = patient.hospitalizations_data[
                                    patient.hospitalizations_data.admission_id == admission_id
                                                            ].to_dict(orient='records')[0]
        self.laboratory_data = patient.laboratory_data[
                                    patient.laboratory_data.admission_id == admission_id
                                                        ]
        self.images_data = patient.images_data[
                                    patient.images_data.admission_id == admission_id
                                                ]
    def get_labos_date_records(self,date):
        
        labos_records = {}
        #Labos 
        mask = self.laboratory_data['labo_date'] == date
        labos = self.laboratory_data[mask]

        labos_records['labos_studies_names'] =  labos.study_description.unique()
        labos_records['labos_set_count'] = labos.labo_id.nunique()
        labos_records['labos_count'] = len(labos.labo_id)
        labos_records['labos_requester_roles'] = labos.requester_role.unique()
        labos_records['labos_requester_roles_count'] = labos.requester_role.nunique()
        labos_records['labos_emergencies'] = labos.emergency.sum()
        labos_records['labos_requesters'] = labos.requester_name.unique()
        labos_records['labos_requesters_count'] = labos.requester_name.nunique()
        
        return labos_records 
    
    def get_images_date_records(self,date):
        
        images_records = {}
        #Labos 
        mask = self.images_data['image_date'] == date
        images = self.images_data[mask]

        images_records['images_count'] =  images.image_id.nunique()
        images_records['images_study_types'] =  images.type_of_service.unique()
        images_records['images_study_types_count'] =  images.type_of_service.nunique()
        images_records['images_studies_names'] =  images.study_description.unique()
        
        images_records['images_requesters'] = images.requester_name.unique()
        images_records['images_requesters_count'] = images.requester_name.nunique()
        images_records['images_requester_roles'] = images.requester_role.unique()
        images_records['images_requester_roles_count'] = images.requester_role.nunique()
        images_records['images_emergencies'] = images.emergency.sum()
    
        return images_records 
        
        
    def get_admission_records(self):
        
        length_of_stay = (self.admission_data['discharge_date']-
                          self.admission_data['admission_date']).days
        
        df = pd.DataFrame(columns=['patient_id',
                                   'admission_id', 
                                   'no._of_admission',
                                   'hosp_day_number', 
                                   'date',
                                   'discharge'])
        
        
        row = {'patient_id': patient.patient_id,
               'admission_id':self.admission_id,
               'no._of_admission': [key.admission_number for key in patient.admission_history.keys() if 
                                        key.admission_id == self.admission_id][0],
               'labos_cumulative': 0,
               'images_cumulative': 0,
               'labos_set_cumulative':0}
          
        for day in range(length_of_stay):
            row['date'] = self.admission_data['admission_date'] + timedelta(day)
            row['hosp_day_number'] =  day
            row['discharge'] =  (True if day == (length_of_stay -1) else False)
            
            #Labos 
            labos_records = self.get_labos_date_records(row['date'])
            row.update(labos_records)
            row['labos_cumulative'] += row['labos_count']
            row['labos_set_cumulative'] += row['labos_set_count']
            
            #Images
            images_records = self.get_images_date_records(row['date'])
            row.update(images_records)
            row['images_cumulative'] += row['images_count']
            
            df = df.append(row, ignore_index=True)

        return df

In [383]:
patient = Patient('654803-6')

In [384]:
patient.load_patient_data(db)

In [385]:
patient.admission_history

{admission_key(admission_number=0, admission_id='466496-1'): <__main__.Admission at 0x11f0e2ba8>}

In [386]:
patient.get_historic_records()

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge,images_count,images_cumulative,images_emergencies,images_requester_roles,images_requester_roles_count,images_requesters,images_requesters_count,images_studies_names,images_study_types,images_study_types_count,labos_count,labos_cumulative,labos_emergencies,labos_requester_roles,labos_requester_roles_count,labos_requesters,labos_requesters_count,labos_set_count,labos_set_cumulative,labos_studies_names
0,654803-6,466496-1,0,0,2017-01-01,True,0.0,0.0,0.0,[],0.0,[],0.0,[],[],0.0,6.0,6.0,1.0,[MEDICO DE PISO],1.0,[PEREZ CARDENAS MARJORIE],1.0,2.0,2.0,"[GLUCOSA EN SANGRE, HEMOGRAMA, IONOGRAMA EN SA..."


In [387]:
db['surgeries'][db['surgeries'].patient_id == '654803-6']

,operating_room,surgery_date,surgery_weekday,surgery_year_month,surgery_scheduled_time,admission_id,surgery_type,G.A.P.,surgery_id,origin,patient_id,gender,age,discharge_type,entity_description,diagnosis,scheduled_surgery,actual_surgery,surgery_physician,dependency,anesthesia_type,ASA,antisepsia,prophylactic_ATB,dosis_mg,bed_request,hemotherapy,hemo_ok,x_ray,cardiologist,supplies,supplies_ok,protocol_no,service_description,surgery_code,sector_bed,specialization,estimated_duration,surgery_startime,surgery_endtime,surgery_delay,hospitalization_date,specialization_code,entry_time,exit_time,anesthesia_startime,anesthesia_endtime,post_surgery_condition,admission_date,admission_time,discharge_date,discharge_time,re_surgery,no_of_surgeries,hips_surgery,injury_condition,pre_surgery_duration,surgery_duration,post_surgery_duration,surgery_prep_duration,re_admission_id,antibiotic,seeding,no._of_assistans,anesthetist_id,no._of_pregnancies,no._births,no._of_cesarean,nulliparous,new_born_id_rel,new_born_admission_id,new_born_weight,new_born_gestation_age,new_born_alive,bact_positive
0,7,2017-01-01,Dom,201701,21:00:00,466496-1,No Programada,False,137489,URG,654803-6,F,43,Alta Medica,OSPIT - O.S. PERS. IND. TEXTIL,APENDICITIS AGUDA,"APENDICECTOMIA NOCTURNA, LAPAROSCOPICA","APENDICECTOMIA NOCTURNA, LAPAROSCOPICA",SOTELO JUAN MANUEL,Interno,General,1.0,IODOPOVIDONA,AMPICILINA-SULBACTAM,1500.0,False,False,False,False,False,False,False,283049,CIRUGIA GENERAL,58052403,0,CIRUGIA GENERAL ADULTOS,00:34:00,21:41:00,22:15:00,30,None,PROC,21:09:00,22:34:00,21:35:00,22:23:00,INT,2017-01-01,14:51:00,2017-01-02,19:08:00,0.0,1,False,Limpia,00:06:00,00:34:00,00:19:00,00:25:00,None,-1.0,-1.0,1,56356,0,0,0,None,NaN,None,NaN,0,None,NaN


In [110]:

        
    @property       
    def gender(self):
        try:
            rv = self.hospitalizations_data['gender'].unique()[0]
        except:
            print('Load patient data before! ')
        
        return rv
            
    @property
    def insurance(self):
        try:
            rv = self.hospitalizations_data['insurance_entity'].unique()[0]
        except:
            print('Load patient data before!')
            
        return rv
    
    @property
    def birth_date(self):
        try: 
            rv = datetime.strptime(self.hospitalizations_data['birth_date'].unique()[0],
                                            '%Y-%m-%d')
        except:
            print('Load patient data before!')
            
        return rv
        
    def get_demographics(self):
        self.demographics = {'gender': self.gender ,
                             'insurance': self.insurance ,
                             'birth_date': self.birth_date}
        
        return self.demographics
        
        
    def get_admissions(self):
        
        if not hasattr(self, 'hospitalizations_data'):
            print('Load patient\'s hospitalizations_data before! ')
            
        self.admissions = {} 
        
        for i,admission_id in enumerate(self.hospitalizations_data['admission_id']):
            
            admission_data = self.hospitalizations_data[(self.hospitalizations_data['admission_id'] == 
                                                         admission_id)].to_dict('list')
            
            admission_data['n_admission'] = i
            
            admission =  Admission(admission_data)
            
            self.admissions[str(admission._admission_id)] = admission
            
    def admission_history(self, admission_id):
        
        admission = self.admissions[admission_id]
        
        length_of_stay = (admission._discharge_date-admission._admission_date).days
        
        df = pd.DataFrame(columns=['patient_id',
                                   'admission_id', 
                                   'no._of_admission',
                                   'hosp_day_number', 
                                   'date',
                                   'discharge',
                                   'no._of_labo'])
        
        for day in range(length_of_stay):
            row = {'patient_id': self.patient_id,
                   'admission_id': admission._admission_id,
                   'no._of_admission': admission._patient_n_admission,
                   'no._of_labo': 
                   'date': admission._admission_date + timedelta(day),
                   'hosp_day_number': day, 
                   'discharge': (True if day == (length_of_stay -1) else False)
                   
            }
            df = df.append(row, ignore_index=True)
            #pd.concat([df,row], ignore_index=True)
            
        return df
        
        
 
        
        
        
        
    

In [111]:
class Admission(Patient):
    def __init__(self, admission_data):
        
        self._admission_id = admission_data['admission_id'][0]
        self._admission_date = datetime.strptime(admission_data['admission_date'][0],
                                                 '%Y-%m-%d')
        self._admission_hour =  admission_data['admission_time'][0]
        self._patient_n_admission = admission_data['n_admission']
        
        self._admission_source = admission_data['Procedencia'][0]
        self._admission_physician = admission_data['admission_physician'][0]
        self._admission_sector = admission_data['SecAdmisión'][0]
        
        
        self._discharge_date = datetime.strptime(admission_data['discharge_date'][0],'%Y-%m-%d')
        self._discharge_hour = admission_data['discharge_time'][0]
        self._discarge_reason = admission_data['Motivo Alta'][0]
        self._pre_discharge_date = admission_data['PreAlta'][0]
        self._discharge_physician = admission_data['Médico Denuncia Egreso'][0]
        
        self._diagnosis = {'administrative_diagnosis_uncod': admission_data['Diagnóstico Administrativo No Codificado'][0],
                           'presumptive_diagnosis_cod': admission_data['CodDiagPresu'][0], 
                           'presumptive_diagnosis_uncod': admission_data['Diagnóstico Presuntivo'][0], 
                           'discharge_diagnosis_cod': admission_data['CodCieDiagEgr'][0], 
                           'discharge_diagnosis_uncod': admission_data['Diagnóstico Egreso'][0],
                           'high_risk_TEP_diagnosis': admission_data['DiagAltoRiesgoTEP'][0],
                           'low_risk_TEP_diagnosis': admission_data['DiagBajoRiesgoTEP'][0]}
        
        self._surgery = admission_data['Quirurg'][0] == 'Sí'
        self._hips_surgery = admission_data['CaderaExpress'][0] == 'Sí'
        self._is_1st_admission = admission_data['Reingreso'][0] == 'No'
        
        self._admission_data = admission_data
        
        
        
    

In [112]:
paciente = Patient('3526141-4')

In [113]:
paciente.load_patient_data('hospitalizations',internaciones)

In [114]:
paciente.hospitalizations_data

,Nro Adm,Nro H.C.,Edad,Sexo,Fec Nac,Entidad,AgrupEntidad,Fec Adm,Hora Adm,AñoAdm,MesAdm,Procedencia,Médico Admisión,SecAdmisión,SecUltimo,CategUlt,Aislación,HabitUlt,CamaUlt,Fec Alta,Hora Alta,AñoAlta,MesAlta,Motivo Alta,PreAlta,1erSecInt,Diagnóstico Administrativo No Codificado,CodDiagPresu,Diagnóstico Presuntivo,CodCieDiagEgr,Diagnóstico Egreso,FecDenunEgre,HoraDenunEgre,Médico Denuncia Egreso,Epicrisis,MédicoEpicrisis,Quirurg,CaderaExpress,Permanencia,Servicio Responsable,Servicio Co-Responsable,FecHorIngGua,Prest.Guardia,Prestac.Guardia,Reingreso,AdmAntReciente,FecAdmAntReciente,FecAltaAntReciente,SecAntReciente,Diagn. Egreso Admisión Anterior Reciente,AmbulanciaEgreso,PesoAlNacer,EdadGestac,PIM2TEP,DiagAltoRiesgoTEP,DiagBajoRiesgoTEP,ARM_TEP,CEC_TEP,SolicDerivación,OrigDerivación,Procedencia.1,DiagnósticoDerivación,AreaDerivación,Notificado,UsuarioDeriv
0,466470-1,3526141-4,79,F,1937-11-15,PAMI UGL VI,PAMI,2017-01-01,02:42:00,2017,Ene,Emergencias,VEDIA VARGAS ANICETA,EME,T16,C,NaN,1634,2,2017-01-04,18:17:00,2017.0,Ene,Alta Medica,NaN,T16,DESHIDRATACION,XX2,No Especifica Codigo,276.51,Deshidratacion,2017-01-04,16:47:41,SIMONE MILENA,Sí,SIMONE MILENA,No,No,3,CLINICA MEDICA,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33624,500538-7,3526141-4,100,F,1917-11-15,PAMI UGL VI,PAMI,2018-02-26,15:24:00,2018,Feb,Emergencias,ROJAS MARIANA GISELLE,EME,T11,C,NaN,1104,2,2018-03-02,17:16:00,2018.0,Mar,Alta Medica,NaN,T11,NEUMOPATIA,786.00,"Alteracion Respiratoria, Sin Especificar",428,Insuficiencia Cardiaca,2018-03-02,14:23:38,ROCCA LUCIANA,Sí,ROCCA LUCIANA,No,No,4,CLINICA MEDICA,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43157.6161,DRA AGRA,DOMICILIO,NEUMOPATIA,INTERNACION GENERAL,DR HEMINSEN,JFOSTER


In [115]:
paciente.get_demographics()

{'birth_date': datetime.datetime(1937, 11, 15, 0, 0),
 'gender': 'F',
 'insurance': 'PAMI UGL VI'}

In [116]:
paciente.demographics

{'birth_date': datetime.datetime(1937, 11, 15, 0, 0),
 'gender': 'F',
 'insurance': 'PAMI UGL VI'}

In [117]:
paciente.get_admissions()

In [118]:
paciente.admissions

{'466470-1': <__main__.Admission at 0x1182fd668>,
 '500538-7': <__main__.Admission at 0x117ff4ba8>}

In [119]:
paciente.admissions['466470-1']._admission_date

datetime.datetime(2017, 1, 1, 0, 0)

In [120]:
paciente.admissions['500538-7']._admission_date

datetime.datetime(2018, 2, 26, 0, 0)

In [121]:
paciente.admissions['466470-1']._admission_date

datetime.datetime(2017, 1, 1, 0, 0)

In [122]:
paciente.admissions['466470-1']._admission_date + timedelta(1)

datetime.datetime(2017, 1, 2, 0, 0)

In [123]:
paciente.admission_history('466470-1')

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge
0,3526141-4,466470-1,0,0,2017-01-01,False
1,3526141-4,466470-1,0,1,2017-01-02,False
2,3526141-4,466470-1,0,2,2017-01-03,True


In [127]:
paciente.admission_history('500538-7')

,patient_id,admission_id,no._of_admission,hosp_day_number,date,discharge
0,3526141-4,500538-7,1,0,2018-02-26,False
1,3526141-4,500538-7,1,1,2018-02-27,False
2,3526141-4,500538-7,1,2,2018-02-28,False
3,3526141-4,500538-7,1,3,2018-03-01,True


In [125]:
paciente.load_patient_data('laboratory',db['labos'])

In [133]:
paciente.laboratory_data.groupby('Fecha').count()

,Nro Vale,Pun,Estado,Hora,HH,AñoMes,Sector,Nro Adm,HistClín,Nombre Paciente,Fec.Adm.,Fec.Alta,Entidad,Nombre Entidad,Nro.Afiliado,GrupoEnt,Urgencia,Mnemo Serv,Servicio,Cod Serv,Nombre del Solicitante,Función del Solicitante,Prestación,Descrip Prestación,CantPrest,Cod Insumo,Descrip Insumo,CantInsumos,EstadoResultado,Observ.Estudio
Fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01/01/2017,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,0,0,0,0,0
02/01/2017,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,0,0,0,0,0
03/01/2017,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,0,0,0,0,0
26/02/2018,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,0,0,0,0,0
27/02/2018,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,0,0,0,0,0


In [95]:
(paciente.admissions['466470-1']._discharge_date-paciente.admissions['466470-1']._admission_date).days

3

In [31]:
for admission_id in paciente.admissions:
    print(paciente.admissions[admission_id]._admission_hour)

02:42:00
15:24:00
